In [ ]:
# Read the list of compounds to test
Target = "CyanoNP2010-2023"

Additional = "CyanoMetDB.csv"


In [ ]:
df_target = pd.read_csv(Target+".csv", sep=",")
m22 = []
for mol in df_target['SMILES']: # for-loop in every row to use the 'smiles' entrees 
    m2 = Chem.MolToInchiKey(Chem.MolFromSmiles(mol)) # produce the INCHIKEY formula
    m22.append(m2) # create a list of INCHIKEY formula 
df_target['InchiKey'] = m22

print(f"List of compounds to be used for this study is now uploaded. It´s size: {df_target.shape}")

In [ ]:
# Define the list of compound of INTEREST
df_CyanoMetDB = pd.read_csv(Additional, sep=",", encoding="latin1")
df_CyanoMetDB.rename(columns={"InChlKey":"InchiKey"}, inplace=True)
print(f"Number of compounds from CHEMBL: {df_CyanoMetDB.shape}")

In [ ]:
# Filter row for empty cells
print(f"Size of the dataframe before filter empty cells: {df_CyanoMetDB.shape}.")
df_CyanoMetDB["SMILES"].replace('',numpy.nan,inplace=True)
df_CyanoMetDB.dropna(subset="SMILES", inplace=True)
# Remove duplicate rows based on the "SMILES" column
df_CyanoMetDB.drop_duplicates(subset=["SMILES"], inplace=True)
print(f"Size of the dataframe after filter empty and duplicated cells: {df_CyanoMetDB.shape}.")

In [ ]:
# Combine both dataframes
merged_df = merge_dataframes([df_target,df_CyanoMetDB], 'InchiKey')
df_Interest = merged_df.loc[merged_df['additional_column_1'] & merged_df['additional_column_2'] == 1]
print(f"Size of the merged structure databases: df_target + df_ChEMBL ({df_target.shape[0]+df_CyanoMetDB.shape[0]}) = {merged_df.shape[0]}")
print(f"Size of the original structure database to study: {df_target.shape[0]}")
print(f"Size of the ChEMBL structure database used as seed for compounds of interest: {df_CyanoMetDB.shape[0]}")
print(f"How many compounds can be promptly identified as compounds of interest (co-occurring in both databases): {df_Interest.shape[0]}")

In [ ]:
# How to get INCHI, INCHIKey, and Exact Mass (molecular weigth) from a series of SMILES
mform = []
mw2 = []

calc = Calculator(descriptors.SLogP.SLogP)

# calculate multiple molecule
mols = [Chem.MolFromSmiles(smi) for smi in merged_df['SMILES']]
df_descriptors = calc.pandas(mols)

for mol in merged_df['SMILES']: # for-loop in every row to use the 'smiles' entrees 
    mols = [Chem.MolFromSmiles(smi) for smi in merged_df['SMILES']]

    mform1 =  CalcMolFormula(Chem.MolFromSmiles(mol)) # calculate the exact molecular weigth 
    mform.append(mform1) # create a list of exact molecular weigth 
    
    mw = Descriptors.ExactMolWt(Chem.MolFromSmiles(mol)) # calculate the exact molecular weigth 
    mw2.append(mw) # create a list of exact molecular weigth     
    
merged_df['MolFormula'] = mform      # include the newly created lists into the DataFrame
merged_df['MolWeight'] = mw2

# save .CSV file with the datafile2 (extended version)
merged_df.to_csv(Project + '_'+ Target +'_extended.csv')

print(f"A new CSV file was saved with all the structures in both databases with additional valuable information, to list: Molecular Formula, Inchi code, Molecular Weight, expected m/z values for [M-H]-, [M+H]+, [M+Na]+, and [M+K]+.")

In [ ]:
from pathlib import Path

# Build queries (as you already do)
q_dict = generate_massql_queries(
    merged_df, ppm=10, intensity_percent=1, separate_adducts=False
)

out_path = Path("MS1_queries_by_compound.txt")
with out_path.open("w", encoding="utf-8") as f:
    for i, (name, q) in enumerate(q_dict.items(), start=1):
        f.write(f"### {i}. {name} ###\n{q}\n\n")  # blank line between queries

In [2]:
import pandas as pd

def merge_dataframes(df_list, key_column):
    # Merge the dataframes using pivot_table
    merged_df = pd.concat(df_list, axis=0, ignore_index=True)
    merged_df = merged_df.pivot_table(index=key_column, aggfunc='first').reset_index()
    # Create the additional columns
    for i, df in enumerate(df_list):
        col_name = f"additional_column_{i+1}"
        merged_df[col_name] = merged_df[key_column].isin(df[key_column]).astype(int)
    # Fill missing values with 0
    merged_df = merged_df.fillna(0)

    return merged_df


import math

# ---- Adduct exact masses (monoisotopic) ----
ADDUCT_MASS = {
    "[M+H]+":   1.007276466,
    "[M+Na]+": 22.989218,     # Na+
    "[M+K]+":  38.963157,     # K+
    "[M+NH4]+":18.033823      # NH4+
}

def _format_mz(x: float, decimals: int = 5) -> str:
    """Format m/z with fixed decimals (MassQL 'OR' list likes clean numbers)."""
    return f"{x:.{decimals}f}"

def compute_adduct_mzs(mono_mass: float, adducts=None, decimals: int = 5):
    """
    Given neutral monoisotopic mass, return dict {adduct: m/z}.
    For the 1+ adducts requested, m/z = M + adduct_mass.
    """
    if adducts is None:
        adducts = list(ADDUCT_MASS.keys())
    out = {}
    for a in adducts:
        out[a] = float(_format_mz(mono_mass + ADDUCT_MASS[a], decimals))
    return out

def massql_query_for_compound(
    name: str,
    mono_mass: float,
    adducts=None,
    ppm: int = 10,
    intensity_percent: int = 1,
    decimals: int = 5,
    separate_adducts: bool = False
):
    """
    Build MassQL query(ies) for one compound.
    - If separate_adducts=False: returns ONE query string with all adduct m/z joined by OR.
    - If separate_adducts=True: returns a dict {adduct: query_string} (one per adduct).
    """
    if adducts is None:
        adducts = ["[M+H]+", "[M+Na]+", "[M+K]+", "[M+NH4]+"]

    mzs = compute_adduct_mzs(mono_mass, adducts=adducts, decimals=decimals)

    if not separate_adducts:
        mz_list = " OR ".join(_format_mz(mzs[a], decimals) for a in adducts)
        q = (
f"# {name}\n"
"QUERY scaninfo(MS1DATA) WHERE\n"
f"MS1MZ=(\n\t{mz_list}\n"
f"):TOLERANCEPPM={ppm}:INTENSITYPERCENT={intensity_percent}"
        )
        return q

    # One query per adduct
    out = {}
    for a in adducts:
        q = (
f"# {name} {a}\n"
"QUERY scaninfo(MS1DATA) WHERE\n"
f"MS1MZ=(\n\t{_format_mz(mzs[a], decimals)}\n"
f"):TOLERANCEPPM={ppm}:INTENSITYPERCENT={intensity_percent}"
        )
        out[a] = q
    return out

def generate_massql_queries(
    df_metadata: pd.DataFrame,
    ppm: int = 10,
    intensity_percent: int = 1,
    decimals: int = 5,
    separate_adducts: bool = False,
    adducts=None
):
    """
    Iterate over df_metadata[['Compound name','MolWeight']] and build queries.
    Returns:
      - If separate_adducts=False: dict {compound_name: query_string}
      - If separate_adducts=True: dict {compound_name: {adduct: query_string}}
    """
    if adducts is None:
        adducts = ["[M+H]+", "[M+Na]+", "[M+K]+", "[M+NH4]+"]

    results = {}
    for _, row in df_metadata.iterrows():
        name = str(row["Compound name"])
        mono = float(row["MolWeight"])
        results[name] = massql_query_for_compound(
            name=name,
            mono_mass=mono,
            adducts=adducts,
            ppm=ppm,
            intensity_percent=intensity_percent,
            decimals=decimals,
            separate_adducts=separate_adducts
        )
    return results
